<a href="https://colab.research.google.com/github/sumandutta8877/DATA-Analytics/blob/suman/DA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data from GitHub

In [131]:
!git clone 'https://github.com/sumandutta8877/Data_DA.git'    # Run this Line for only one time

fatal: destination path 'Data_DA' already exists and is not an empty directory.


In [141]:
import pandas as pd
import numpy as np
df_train = pd.read_csv('/content/Data_DA/Data/Train_data.csv')
df_test = pd.read_csv('/content/Data_DA/Data/Test_data.csv')
df_sample = pd.read_csv('/content/Data_DA/Data/Sample_submission.csv')
#print(df_train.head)
#print(df_test.head)
#print(df_sample.head)
df1 = df_train            # df1 is the training dataframe
df2 = df_test             # df2 is the test dataframe
ID = df_test['id'].to_numpy()
df_id = pd.DataFrame(ID, columns = ['id'])
df_id.to_csv(r'/content/Data_DA/Data/Sample_output.csv',index=False)

# Start Processing

In [142]:
df1['mode'] = df1['mode'].apply(lambda x: 2 if x == 'Major' else 1)
df1['explicit'] = df1['explicit'].apply(lambda x: 0.2 if x == 'No' else 1.2)
df1['id'] = 1
df2['mode'] = df2['mode'].apply(lambda x: 2 if x == 'Major' else 1)
df2['explicit'] = df2['explicit'].apply(lambda x: 0.2 if x == 'No' else 1.2)
df2['id'] = 1
df1.loc[(df1['popularity'] == 'very low'),'ratings'] = 1
df1.loc[(df1['popularity'] == 'low'),'ratings'] = 2
df1.loc[(df1['popularity'] == 'average'),'ratings'] = 3
df1.loc[(df1['popularity'] == 'high'),'ratings'] = 4
df1.loc[(df1['popularity'] == 'very high'),'ratings'] = 5
M_vt = df1['ratings'].to_numpy()
del df1['ratings']
del df1['popularity']
del df1['year']
del df2['year']
df = df1['id']
df['temp1'] = df1['release_date'].str[0:2].astype(int)
df['temp2'] = df1['release_date'].str[3:5].astype(int)
df['temp3'] = df1['release_date'].str[6:10].astype(int)
df1['loudness'] = -1.0*df1['loudness']
df1['release_date'] = 737300 - (df['temp1']+30.45*df['temp2']+365*df['temp3'])  # number of days old from today
df = df2['id']
df['temp1'] = df2['release_date'].str[0:2].astype(int)
df['temp2'] = df2['release_date'].str[3:5].astype(int)
df['temp3'] = df2['release_date'].str[6:10].astype(int)
df2['loudness'] = -1.0*df2['loudness']
df2['release_date'] = 737300 - (df['temp1']+30.45*df['temp2']+365*df['temp3'])  # number of days old from today

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [143]:
M_tr = df1.to_numpy()
M_ts = df2.to_numpy()
W = numpy.matmul((numpy.matmul(np.linalg.inv(numpy.matmul(np.transpose(M_tr),M_tr)),np.transpose(M_tr))),M_vt)
Y_output = numpy.matmul(M_ts,W)
for i in range(len(Y_output)):
  if Y_output[i]<=1.0:
    Y_output[i]=1.0
  elif Y_output[i]<=2.0:
    Y_output[i]=2.0
  elif Y_output[i]<=3.0:
    Y_output[i]=3.0
  elif Y_output[i]<=4.0:
    Y_output[i]=4.0
  else:
    Y_output[i]=5.0
print(Y_output)
df = pd.DataFrame(Y_output, columns = ['ratings'])
df.loc[(df['ratings'] == 1.0),'popularity'] = 'very low'
df.loc[(df['ratings'] == 2.0),'popularity'] = 'low'
df.loc[(df['ratings'] == 3.0),'popularity'] = 'average'
df.loc[(df['ratings'] == 4.0),'popularity'] = 'high'
df.loc[(df['ratings'] == 5.0),'popularity'] = 'very high'
del df['ratings']

[4. 3. 3. ... 3. 1. 4.]


In [144]:
df_id=pd.read_csv(r'/content/Data_DA/Data/Sample_output.csv')
data = [df_id['id'],df['popularity']]
headers = ['id','popularity']
df_output = pd.concat(data, axis=1, keys=headers)
df_output.to_csv(r'/content/Data_DA/Data/Sample_output.csv',index=False)